In [249]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta
import datetime
import numpy as np
from functools import reduce
import IPython
from IPython.display import display, HTML
import re
import warnings
warnings.filterwarnings('ignore')
# warnings.filterwarnings(action='once')

In [3]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [4]:
import cufflinks as cf
from plotly.offline import download_plotlyjs, plot,iplot
import plotly.tools as tls
cf.go_offline()

In [5]:
import numpy.ma as ma

In [6]:
import plotly.figure_factory as ff

#### 1.1 Covid recoveries data

In [7]:
metrics_recoveries = pd.read_excel('./Data/Covid_Metrics.xlsx', sheet_name='Recoveries')

In [8]:
metrics_recoveries.columns.nunique()

343

In [9]:
pd.date_range('2020-03-31','2020-12-31',periods=1).to_list()

[Timestamp('2020-03-31 00:00:00')]

In [10]:
datetym = pd.Series(pd.date_range('2020-03-31','2020-12-31'))

In [11]:
# metrics_recoveries.drop(metrics_recoveries.iloc[:,2:63],inplace=True,axis=1)

In [12]:
newdf = metrics_recoveries.copy()

In [13]:
newdf.drop(newdf.iloc[:,2:63],inplace=True,axis=1)

In [14]:
newdf.drop(newdf.iloc[:,-4:],inplace=True,axis=1)

In [15]:
# newdf.columns.unique().to_list()

In [16]:
new_names= datetym.dt.strftime('%m-%d').to_list()

In [17]:
newdf.rename(columns=dict(zip(newdf.columns[2:],new_names)),inplace=True)

In [18]:
metrics_recoveries = newdf

In [19]:
state_list = ['Andhra Pradesh', 'Arunachal Pradesh','Assam', 'Bihar', 'Chattisgarh', 'Goa', 'Gujarat',
       'Haryana', 'Himachal Pradesh',
       'Jharkhand', 'Karnataka', 'Kerala',
       'Madhya Pradesh','Maharashtra', 'Manipur', 'Meghalaya', 'Mizoram', 'Nagaland',
       'Odisha', 'Punjab','Rajasthan', 'Sikkim', 'Tamil Nadu', 'Telangana', 'Tripura',
       'Uttar Pradesh', 'Uttarakhand', 'West Bengal',
       'Andaman & Nicobar Islands', 'Chandigarh',
       'Dadra and Nagar Haveli and Daman and Diu', 'Jammu & Kashmir',
       'Ladakh', 'Lakshadweep', 'Delhi', 'Puducherry']

In [20]:
metrics_recoveries = metrics_recoveries[metrics_recoveries['State'].isin(state_list)]

#### 1.2 Covid total cases data

In [21]:
metrics_total_cases = pd.read_excel('./Data/Covid_Metrics.xlsx', sheet_name='Total Cases')
metrics_total_cases.drop(metrics_total_cases.iloc[:,2:63],inplace=True,axis=1)
metrics_total_cases.drop(metrics_total_cases.iloc[:,-4:],inplace=True,axis=1)
metrics_total_cases.rename(columns=dict(zip(metrics_total_cases.columns[2:],new_names)),inplace=True)
metrics_total_cases = metrics_total_cases[metrics_total_cases['State'].isin(state_list)]

#### 1.3 Covid deaths data

In [22]:
metrics_deaths = pd.read_excel('./Data/Covid_Metrics.xlsx', sheet_name='Deaths')
metrics_deaths.drop(metrics_deaths.iloc[:,2:63],inplace=True,axis=1)
metrics_deaths.drop(metrics_deaths.iloc[:,-4:],inplace=True,axis=1)
metrics_deaths.rename(columns=dict(zip(metrics_deaths.columns[2:],new_names)),inplace=True)
metrics_deaths = metrics_deaths[metrics_deaths['State'].isin(state_list)]

#### 1.4 Covid active cases data

In [23]:
metrics_active_cases = pd.read_excel('./Data/Covid_Metrics.xlsx', sheet_name='Active Cases')
metrics_active_cases.drop(metrics_active_cases.iloc[:,2:63],inplace=True,axis=1)
metrics_active_cases.drop(metrics_active_cases.iloc[:,-4:],inplace=True,axis=1)
metrics_active_cases.rename(columns=dict(zip(metrics_active_cases.columns[2:],new_names)),inplace=True)
metrics_active_cases = metrics_active_cases[metrics_active_cases['State'].isin(state_list)]

### 2. Covid testing data

In [24]:
testing_data = pd.read_excel('./Data/Testing_Metrics.xlsx', sheet_name='Testing Data')
testing_data.drop(testing_data.iloc[:,1:29],inplace=True,axis=1)
testing_data.drop(testing_data.iloc[:,-6:],inplace=True,axis=1)
testing_data.rename(columns=dict(zip(testing_data.columns[1:],new_names)),inplace=True)

In [25]:
statelist=testing_data.State[::4][:(len(testing_data.State[::4])-2)].to_list()
statelist = [state for state in statelist for i in range(4)]

In [26]:
[statelist.append('All INDIA') for i in range(8)]

[None, None, None, None, None, None, None, None]

In [27]:
testing_data.insert(loc=0,column='State_index',value=statelist)

In [28]:
# display(HTML(testing_data.to_html()))

In [29]:
testdata = testing_data.copy()
testdata= testdata.groupby([testdata.State_index, testdata.State]).apply(lambda x: x[:])

##### 2.1 Testing data with State_index and State as multiindices

In [30]:
testdata.drop(['State_index','State'],axis=1, inplace=True)

In [31]:
testdata1 = testing_data.groupby([testing_data.State,testing_data.State_index]).apply(lambda x: x[:])

### 3. ICMR_labs_statewise

In [32]:
labs = pd.read_excel('./Data/icmr_labs_statewise.xlsx')

In [33]:
private_fac = []
for col in labs.columns.tolist():
    if col.endswith("1"):
        private_fac.append(col)

In [34]:
private = labs[private_fac]
private['Date'] = labs['Unnamed: 0']

In [35]:
cols = private.columns.tolist()
cols= cols[-1:] + cols[:-1]
private = private[cols]

In [36]:
government = labs.copy()
government = government[government.columns.difference(private_fac)]
government = government.rename(columns={'Unnamed: 0':'Date'})
government['Unnamed: 1'] = labs['Unnamed: 1']

In [37]:
government = government[['Date','Unnamed: 1','Andaman and Nicobar Islands','Andhra Pradesh','Arunachal Pradesh','Assam','Bihar','Chandigarh',
 'Chhattisgarh', 'Dadra & Nagar Haveli','Delhi','Goa','Gujarat','Haryana','Himachal Pradesh','Jammu and Kashmir','Jharkhand','Karnataka',
 'Kerala','Ladakh','Lakshwadeep','Madhya Pradesh','Maharashtra','Manipur','Meghalaya','Mizoram','Nagaland','Odisha','Puducherry',
 'Punjab','Rajasthan','Sikkim','Tamil Nadu','Telanagana','Tripura','Uttar Pradesh','Uttarakhand','West Bengal','Mobile Vans','Duplicates',
 'Total','Changes']]

In [38]:
# Dropping the first row from government and private dataframes
government = government[1:]
private = private[1:]
datelist = government.Date.tolist()

In [39]:
natlist = []
for i in range(len(datelist)):
    if type(datelist[i]) == pd._libs.tslibs.nattype.NaTType:
        natlist.append(i)

In [40]:
for i in range(len(datelist)):
    if type(datelist[i]) == pd._libs.tslibs.nattype.NaTType:
        datelist[i] = datelist[i-1]

In [41]:
comparisondf = pd.DataFrame({'orignat': government.Date.tolist(),
                             'datelist': datelist
                            })

In [42]:
# Setting date column to be datelist for government and private
government['Date'] = datelist
private['Date'] = datelist

government = government.rename(columns={'Unnamed: 1':'Tests'})
private = private.rename(columns={'Unnamed: 1':'Tests'})

In [43]:
# Renaming private dataframe columns on the lines of government df for further processing of addition across columns to get a total test dataset
private.rename(columns=dict(zip(private.columns.tolist(),government.columns.tolist())),inplace=True)

In [44]:
# Groupby govt and pvt dataframes and setting Date and Tests as multiindexes 
gov_grp = government.groupby([government.Date,government.Tests]).apply(lambda x: x[:]).set_index(['Date','Tests'])
pvt_grp = private.groupby([private.Date,private.Tests]).apply(lambda x: x[:]).set_index(['Date','Tests'])

In [45]:
totaltests = pd.DataFrame(index=gov_grp.index, columns=gov_grp.columns)
totaltests = totaltests.fillna(0)
for colG in gov_grp.columns.tolist():
    for colP in pvt_grp.columns.tolist():
        if colG == colP:
            totaltests[colG] = gov_grp[colG].add(pvt_grp[colP],fill_value=0)

In [46]:
dftest = pd.DataFrame(index=totaltests.index)
dftest = dftest.droplevel(level='Tests')

In [47]:
for col in totaltests.columns:
    dftest[col] = totaltests.groupby(['Date','Tests'])[col].sum().unstack(level=['Tests']).sum(axis=1)
    

In [48]:
totaldf = dftest.copy()

In [49]:
totallist = []
for i in range(len(dftest)):
    totallist.append('All3')

In [50]:
totaldf['Tests'] = totallist

colz = totaldf.columns.tolist()
totaldf = totaldf[colz[-1:] + colz[:-1]]
totaldf = totaldf.drop_duplicates()
totaltests.reset_index(inplace=True, level='Tests')

In [51]:
# private.groupby([private.Date,private.Tests]).apply(lambda x: x[:]).set_index(['Date','Tests'])

In [52]:
totaldf1 = pd.concat([totaltests,totaldf]).drop_duplicates().reset_index()
totaldf = totaldf1.groupby(['Date','Tests']).apply(lambda x: x[:]).set_index(['Date','Tests'])

### 4. Mobility data

<strong>Mobility data</strong>

In [53]:
mobility = pd.read_csv('./Data/India_State_District_Mobility_Final.csv')

In [54]:
mobility = mobility[['sub_region_1', 'sub_region_2','iso_3166_2_code','date',
                     'retail_and_recreation_percent_change_from_baseline',
                     'grocery_and_pharmacy_percent_change_from_baseline','parks_percent_change_from_baseline',
                     'transit_stations_percent_change_from_baseline','workplaces_percent_change_from_baseline',
                     'residential_percent_change_from_baseline'
                    ]]

In [55]:
mobilitycopy = mobility.copy()
mobilitycopy = mobilitycopy.dropna(subset = ['sub_region_1','sub_region_2']).reset_index(drop=True)

In [56]:
newdict = {}
for state in mobilitycopy.sub_region_1.unique().tolist():
    newdict[state] = len(mobilitycopy[mobilitycopy.sub_region_1 == state])*100/len(mobility[mobility.sub_region_1 == state]['sub_region_1'].notnull())

### 5. Infrastructure and behaviour data

In [57]:
district_key = pd.read_stata('./Data/lgd_district_key.dta')

In [58]:
district_key = district_key.rename(columns={'lgd_district_id':'pc11_district_id','lgd_state_id':'pc11_state_id'})

#### 5.1 Hospitals

In [59]:
hospitals_dist = pd.read_csv('./Data/Hospital/dlhs4_hospitals_dist.csv')

#### 5.2 Migration

In [60]:
migration = pd.read_csv('./Data/Migration/district_migration.csv')

#### 5.3 Health infra

In [61]:
health_infra = pd.read_csv('./Data/NFHS/ddl_health_infra_2011.csv')

#### 5.4 NFHS 2011

In [62]:
nfhs = pd.read_csv('./Data/NFHS/ddl_nfhs_2011.csv')

#### 5.5 Pollution

In [63]:
pollution = pd.read_csv('./Data/NFHS/ddl_pollution_sedac_2011.csv')

#### 5.6 Demography

In [64]:
demo_agebins = pd.read_csv('./Data/Demography/age_bins_district_t.csv')

In [65]:
demo_totalpop = pd.read_csv('./Data/Demography/dem_district.csv')

In [66]:
demo_totalpop = demo_totalpop.rename(columns={'lgd_district_id':'pc11_district_id','lgd_state_id':'pc11_state_id'})

In [67]:
nfhs = nfhs.drop(nfhs[nfhs.pc11_state_id == 'Uttarakhand'].index)
nfhs['pc11_district_id'] = nfhs['pc11_district_id'].fillna(0).astype(np.int64) 
nfhs['pc11_state_id'] = nfhs['pc11_state_id'].fillna(0).astype(np.int64) 

In [68]:
health_infra = health_infra.drop(health_infra[health_infra.pc11_state_id == 'Uttarakhand'].index)
health_infra['pc11_district_id'] = health_infra['pc11_district_id'].fillna(0).astype(np.int64) 
health_infra['pc11_state_id'] = health_infra['pc11_state_id'].fillna(0).astype(np.int64) 

In [69]:
pollution = pollution.drop(pollution[pollution.pc11_state_id == 'Uttarakhand'].index)
pollution['pc11_district_id'] = pollution['pc11_district_id'].fillna(0).astype(np.int64) 
pollution['pc11_state_id'] = pollution['pc11_state_id'].fillna(0).astype(np.int64) 

In [70]:
demo_agebins = demo_agebins.drop(demo_agebins[demo_agebins.pc11_state_id == 'Uttarakhand'].index)
demo_agebins['pc11_district_id'] = demo_agebins['pc11_district_id'].fillna(0).astype(np.int64) 
demo_agebins['pc11_state_id'] = demo_agebins['pc11_state_id'].fillna(0).astype(np.int64) 

In [71]:
demo_totalpop['pc11_district_id'] = demo_totalpop['pc11_district_id'].fillna(0).astype(np.int64) 
demo_totalpop['pc11_state_id'] = demo_totalpop['pc11_state_id'].fillna(0).astype(np.int64) 

In [72]:
migration = migration.rename(columns={'lgd_district_id':'pc11_district_id','lgd_state_id':'pc11_state_id'})

In [73]:
mergedf2 = nfhs.merge(migration,how='outer',on=['pc11_state_id','pc11_district_id']).merge(health_infra,how='outer',on=['pc11_state_id','pc11_district_id']).merge(pollution,how='outer',on=['pc11_state_id','pc11_district_id']).merge(demo_agebins,how='outer',on=['pc11_state_id','pc11_district_id']).merge(demo_totalpop,how='outer',on=['pc11_state_id','pc11_district_id'])

In [74]:
merged_df = mergedf2[['pc11_district_id','pc11_district_name','pc11_state_id','pc11_state_name','hand_wash_water','hand_wash_soap','mem_per_room',
  'dum_65p','ex_diab_broad_ind_fem','ex_htn_broad_ind_fem',
 'bmigrt27_5_fem','csmoke_fem',
  #migration data
  'inltmigrationrate', 'inltmigrationshare','instmigrationrate',
 'instmigrationshare','outltmigrationrate', 'outltmigrationshare','outstmigrationrate', 'outstmigrationshare',
 #nfhs
 'pr_2011_phc', 'pr_2011_chc', 'pr_2011_sub', 'pr_2011_dish', 'pr_2011_sth',
  #demography
  'pc11_pca_tot_t','pc11_pdensity', 'pc11_pca_tot_p',
  #age bins
  'age_70_t_share','age_75_t_share','age_80_t_share','age_65_t_share','age_60_t_share',
  #pollution
  'mean_pollution'
 ]]

In [75]:
merged_df['60plus_share'] = merged_df['age_70_t_share']+merged_df['age_60_t_share']+merged_df['age_65_t_share']+merged_df['age_75_t_share']+merged_df['age_80_t_share']

In [76]:
# Selecting states which are in the state_list
metrics_total_cases = metrics_total_cases[metrics_total_cases['State'].isin(state_list)]

## Phase wise analysis of top districts

### Phase 1  - Active cases (Apr-May-Jun 2020)

In [77]:
phase1 = metrics_active_cases[metrics_active_cases['District'].isin(metrics_active_cases[(metrics_active_cases.State != metrics_active_cases.District) & (metrics_active_cases.District != 'Unknown') & (metrics_active_cases.District != 'Unknown (migrants and foreign returnees)')].sort_values(by=['06-30'],ascending=False).head(10)['District'].tolist())].loc[:,'State':'06-30'].query('State != "Bihar"')

In [78]:
phase1 = phase1[['District','04-01','04-10','04-20','04-30','05-10','05-20','05-30',
       '06-09','06-19','06-29']].melt(id_vars =['District'],
                                                        var_name = 'Phase1_Date',
                                                        value_name = 'Active_Case'
                                                       )

phase1['Phase1_Date'] =  '2020-'+ phase1.Phase1_Date 
phase1['Phase1_Date'] =  pd.to_datetime(phase1.Phase1_Date,format='%Y-%m-%d')
phase1city = phase1.District.unique().tolist()
phase1city.extend(['Bangalore Urban','Mumbai'])

In [79]:
phase1copy = phase1.copy(deep=True)
phase1copy = phase1copy.set_index('Phase1_Date')

##### Mobility data for Phase 1

In [80]:
phase1mobility = mobilitycopy[mobilitycopy.sub_region_2.isin(phase1city) & (mobilitycopy.sub_region_1 != 'Bihar')][['sub_region_2','date',
                                                         'grocery_and_pharmacy_percent_change_from_baseline']].query('(date>="2020-04-01") & (date<="2020-06-30")').reset_index(drop=True)
phase1mobility['sub_region_2'] = phase1mobility['sub_region_2'].replace({'Bangalore Urban':'Bengaluru Urban','Mumbai':'Mumbai City'})
phase1mobility['date'] =  pd.to_datetime(phase1mobility.date,format='%Y-%m-%d')
phase1mobility = phase1mobility.rename(columns={'sub_region_2':'District','date':'Phase1_Date'})
phase1mobility = phase1mobility[phase1mobility.Phase1_Date.isin(phase1.Phase1_Date.unique().tolist())].reset_index(drop=True)
phase1mobility = phase1mobility.merge(phase1,how='left',on=['District','Phase1_Date']).reset_index(drop=True)

In [81]:
df1 = merged_df[merged_df.pc11_district_name.isin(['Aurangabad', 'Chennai', 'Hyderabad', 'Thane', 'Pune','Ahmadabad','Bangalore','Mumbai'])][['pc11_state_name','pc11_district_name','pc11_pca_tot_t']].query('pc11_state_name != "Bihar"').reset_index(drop=True)
df1 = df1.rename(columns={'pc11_district_name':'District','pc11_pca_tot_t':'total_population'})
df1['District'] = df1['District'].replace({'Bangalore':'Bengaluru Urban','Mumbai':'Mumbai City', 'Ahmadabad':'Ahmedabad'})
df1['total_population'] = df1['total_population'].astype(np.int64)
phase1mobility = phase1mobility.merge(df1,how='left',on=['District']).reset_index(drop=True)
phase1mobility['Phase1_Date'] = phase1mobility.Phase1_Date.dt.strftime('%Y-%m-%d')
phase1mobility = phase1mobility.dropna(subset=['total_population'])
del df1

##### Testing data for Phase 1

In [82]:
phase1dates = ['05-01','05-07','05-14','05-21','05-28','06-04','06-11','06-19','06-26','06-30']

In [83]:
# Dataframe for Percentage Positive values across states for the entire time period
percpositive = testdata.loc[pd.IndexSlice[:,'Percentage Positive (cumulative)'],:].reset_index().drop(['State','level_2'],axis=1).rename(columns={'State_index':'State'}).set_index('State').transpose()

In [84]:
# Transpose of Dataframe for Percentage Positive values across states for the entire time period
percpositiveT = testdata.loc[pd.IndexSlice[:,'Percentage Positive (cumulative)'],:].reset_index().drop(['State','level_2'],axis=1).rename(columns={'State_index':'State'}).set_index('State')

In [85]:
# Dataframe for tests performed across states for the entire time period
testperf = testdata.loc[pd.IndexSlice[:,'No. of Test Performed(cumulative)'],:].reset_index().drop(['State','level_2'],axis=1).rename(columns={'State_index':'State'}).set_index('State').transpose()

In [86]:
# Transpose of Dataframe for tests performed across states for the entire time period
testperfT = testdata.loc[pd.IndexSlice[:,'No. of Test Performed(cumulative)'],:].reset_index().drop(['State','level_2'],axis=1).rename(columns={'State_index':'State'}).set_index('State')

###### Top 5 states with test positivity cumulative till 30th June (end of phase 1)

In [87]:
df1 = percpositiveT.loc[percpositiveT.loc[:,'05-01':'06-30']['06-30'].sort_values(ascending=False).head(6).to_frame().index.tolist(), :].loc[:,'05-01':'06-30']

In [88]:
df1_alt = df1.copy()
# sns.heatmap(df1_alt, cmap="YlGnBu", linewidths=0.5)

In [89]:
df1 = df1.reset_index()
df1 = df1.melt(id_vars=['State'],var_name='Date',value_name='Test_Positivity')
# df1['Test_Positivity'] = df1.Test_Positivity.astype(int)
df1['Date'] =  '2020-'+ df1.Date
df1['Date'] =  pd.to_datetime(df1.Date,format='%Y-%m-%d')

In [90]:
gujarat = testdata.loc[("Gujarat")].loc[:,phase1dates].loc['No. of Test Performed(cumulative)']
karnataka = testdata.loc[("Karnataka")].loc[:,phase1dates].loc['No. of Test Performed(cumulative)']
maharashtra = testdata.loc[("Maharashtara")].loc[:,phase1dates].loc['No. of Test Performed(cumulative)']
tamilnadu = testdata.loc[("Tamil Nadu")].loc[:,phase1dates].loc['No. of Test Performed(cumulative)']
telangana = testdata.loc[("Telangana")].loc[:,phase1dates].loc['No. of Test Performed(cumulative)']

In [91]:
gujarat['State'] = ['Gujarat']
karnataka['State'] = ['Karnataka']
maharashtra['State'] = ['Maharashtara']
tamilnadu['State'] = ['Tamil Nadu']
telangana['State'] = ['Telangana']

In [92]:
gujarat = gujarat[gujarat.columns.tolist()[-1:] + gujarat.columns.tolist()[:-1]].reset_index(drop=True)
karnataka = karnataka[karnataka.columns.tolist()[-1:] + karnataka.columns.tolist()[:-1]].reset_index(drop=True)
maharashtra = maharashtra[maharashtra.columns.tolist()[-1:] + maharashtra.columns.tolist()[:-1]].reset_index(drop=True)
tamilnadu = tamilnadu[tamilnadu.columns.tolist()[-1:] + tamilnadu.columns.tolist()[:-1]].reset_index(drop=True)
telangana = telangana[telangana.columns.tolist()[-1:] + telangana.columns.tolist()[:-1]].reset_index(drop=True)

In [93]:
totaldf1= totaldf[['Bihar','Gujarat','Karnataka','Maharashtra','Tamil Nadu','Telanagana']].loc['2020-05-04':'2020-06-29',:].xs('All3',level=1).reset_index(level=0)

In [94]:
melt1 = totaldf1.melt(id_vars=['Date'],var_name='State',value_name='Testing_Labs')
melt1['Date'] = melt1['Date'].dt.strftime('%Y/%m/%d')
melt1['Testing_Labs'] = melt1.Testing_Labs.astype(int)

In [95]:
melt2 = totaldf1.melt(id_vars=['Date'],var_name='State',value_name='Testing_Labs')
# melt1['Date'] = melt1['Date'].dt.strftime('%Y/%m/%d')
melt2['Testing_Labs'] = melt2.Testing_Labs.astype(int)

In [96]:
# sns.relplot(x="Date", y="Testing_Labs",
# data=melt1,
# kind="line",
# #style="sub_region_2",
# hue="State")

##### Phase 1 State infrastructure

In [97]:
select_df = merged_df[['pc11_district_id','pc11_district_name','pc11_state_id','pc11_state_name','hand_wash_soap','mem_per_room',
'dum_65p','ex_diab_broad_ind_fem','ex_htn_broad_ind_fem',
 'bmigrt27_5_fem','csmoke_fem','instmigrationrate','instmigrationshare','outstmigrationrate', 'outstmigrationshare',
                       'inltmigrationrate','inltmigrationshare','outltmigrationrate', 'outltmigrationshare',
    'pr_2011_phc','pr_2011_dish','pc11_pca_tot_t','age_70_t_share','age_80_t_share','age_65_t_share','age_60_t_share','mean_pollution','60plus_share']]

In [98]:
# display(HTML(select_df.to_html()))

In [99]:
citypresent = []
for i in range(len(phase1city)):
    if phase1city[i] in select_df.pc11_district_name.unique().tolist():
        citypresent.append(phase1city[i])
        

In [100]:
citypresent

['Aurangabad', 'Pune', 'Thane', 'Chennai', 'Hyderabad', 'Mumbai']

In [101]:
city1 = ['Aurangabad', 'Mumbai', 'Pune', 'Thane', 'Chennai', 'Hyderabad','Bangalore','Ahmadabad']

In [102]:
bubble1= select_df[select_df.pc11_district_name.isin(['Aurangabad', 'Mumbai', 'Pune', 'Thane', 'Chennai', 'Hyderabad','Bangalore','Ahmadabad'])]

In [103]:
bubble1.drop(bubble1[(bubble1.pc11_district_name == 'Aurangabad') & (bubble1.pc11_state_name == 'Bihar')].index, inplace=True)

### Phase 2  - Active cases (Jul-Aug-Sep 2020)

In [104]:
phase2dates = ['07-01', '07-11','07-21','07-31','08-11','08-21','08-31','09-10','09-21','09-30']

In [105]:
phase2 = metrics_active_cases[metrics_active_cases['District'].isin(metrics_active_cases[(metrics_active_cases.State != metrics_active_cases.District) & (metrics_active_cases.District != 'Unknown') & (metrics_active_cases.District != 'Unknown (migrants and foreign returnees)')].loc[:,['State','District','07-01', 
'07-11','07-21','07-31','08-11','08-21','08-31','09-10','09-21','09-30']].sort_values(by=['09-30'],ascending=False).head(10)['District'].tolist())].loc[:,['State','District','07-01', '07-11','07-21',
 '07-31','08-11','08-21','08-31','09-10','09-21','09-30']]

In [106]:
phase2 = phase2[['District','07-01', '07-11','07-21','07-31','08-11','08-21','08-31','09-10','09-21','09-30']].melt(id_vars =['District'],
                                                        var_name = 'Phase2_Date',
                                                        value_name = 'Active_Case'
                                                       )

In [107]:
phase2['Phase2_Date'] =  '2020-'+ phase2.Phase2_Date 
phase2['Phase2_Date'] =  pd.to_datetime(phase2.Phase2_Date,format='%Y-%m-%d')

In [108]:
# display(HTML(phase2.to_html()))

In [109]:
phase2city = phase2.District.unique().tolist()

In [110]:
# merged_df[merged_df.pc11_district_name.isin(phase2city)]['pc11_state_name'].unique()
phase2states = ['Chhattisgarh', 'Maharashtra', 'Andhra Pradesh', 'Kerala','Tamil Nadu']

In [111]:
phase2copy = phase2.copy(deep=True)

In [112]:
phase2copy = phase2copy.set_index('Phase2_Date')

In [113]:
mobilitycopy2 = mobilitycopy.copy(deep=True)

In [114]:
mobilitycopy2['sub_region_2'] = mobilitycopy2['sub_region_2'].replace({'Bangalore Urban':'Bengaluru Urban','Mumbai':'Mumbai City'})
phase2mobility = mobilitycopy2[mobilitycopy2.sub_region_2.isin(phase2city)][['sub_region_2','date',
                                                         'grocery_and_pharmacy_percent_change_from_baseline']].query('(date>="2020-06-01") & (date<="2020-09-30")').reset_index(drop=True)

phase2mobility['date'] =  pd.to_datetime(phase2mobility.date,format='%Y-%m-%d')
phase2mobility = phase2mobility.rename(columns={'sub_region_2':'District','date':'Phase2_Date'})
phase2mobility = phase2mobility[phase2mobility.Phase2_Date.isin(phase2.Phase2_Date.unique().tolist())].reset_index(drop=True)
phase2mobility = phase2mobility.merge(phase2,how='left',on=['District','Phase2_Date']).reset_index(drop=True)

merged_df_2 = merged_df.copy(deep=True)
merged_df_2['pc11_district_name'] = merged_df_2['pc11_district_name'].replace({'Bangalore':'Bengaluru Urban','Mumbai':'Mumbai City'})
df_2 = merged_df_2[merged_df_2.pc11_district_name.isin(phase2city)][['pc11_state_name','pc11_district_name','pc11_pca_tot_t']].reset_index(drop=True)
df_2 = df_2.rename(columns={'pc11_district_name':'District','pc11_pca_tot_t':'total_population'})
# df1['District'] = df1['District'].replace({'Bangalore':'Bengaluru Urban','Mumbai':'Mumbai City', 'Ahmadabad':'Ahmedabad'})
df_2.loc[df_2[df_2['District'] == 'Thiruvananthapuram'].index, 'total_population'] = 1679754.0
df_2['total_population'] = df_2['total_population'].astype(np.int64)

phase2mobility = phase2mobility.merge(df_2,how='left',on=['District']).reset_index(drop=True)
phase2mobility['Phase2_Date'] = phase2mobility.Phase2_Date.dt.strftime('%Y-%m-%d')
phase2mobility = phase2mobility.dropna(subset=['total_population'])

del df_2
del merged_df_2

##### Taking data for second phase for percent postivity

In [115]:
df_2alt = percpositiveT.loc[percpositiveT.loc[:,'07-01':'09-30']['09-30'].sort_values(ascending=False).head(5).to_frame().index.tolist(), :].loc[:,'07-01':'09-30']

In [116]:
df_2 = percpositiveT.loc[percpositiveT.loc[:,'07-01':'09-30']['09-30'].sort_values(ascending=False).head(5).to_frame().index.tolist(), :].loc[:,'07-01':'09-30']

In [117]:
all_India = percpositiveT.iloc[percpositiveT.loc[:,'07-01':'09-30'].index == 'All INDIA'].loc[:,'07-01':'09-30']
df_2 = all_India.append(df_2)

In [118]:
df_2 = df_2.reset_index()
df_2 = df_2.melt(id_vars=['State'],var_name='Date',value_name='Test_Positivity')
# df1['Test_Positivity'] = df1.Test_Positivity.astype(int)
df_2['Date'] =  '2020-'+ df_2.Date
df_2['Date'] =  pd.to_datetime(df_2.Date,format='%Y-%m-%d')

##### Phase 2 - testing labs 

In [119]:
phase2states

['Chhattisgarh', 'Maharashtra', 'Andhra Pradesh', 'Kerala', 'Tamil Nadu']

In [120]:
totaldf_2= totaldf[phase2states].loc['2020-07-06':'2020-09-28',:].xs('All3',level=1).reset_index(level=0)

In [121]:
melt_2 = totaldf_2.melt(id_vars=['Date'],var_name='State',value_name='Testing_Labs')
melt_2['Date'] = melt_2['Date'].dt.strftime('%Y/%m/%d')
melt_2['Testing_Labs'] = melt_2.Testing_Labs.astype(int)

In [122]:
melt_2 = totaldf_2.melt(id_vars=['Date'],var_name='State',value_name='Testing_Labs')
melt_2['Testing_Labs'] = melt_2.Testing_Labs.astype(int)

##### Phase 2 state infrastructure

In [123]:
select_df_2 = select_df.copy(deep=True)
select_df_2['pc11_district_name'] = select_df_2['pc11_district_name'].replace({'Bangalore':'Bengaluru Urban','Mumbai':'Mumbai City'})

In [124]:
bubble_2 = select_df_2[select_df_2.pc11_district_name.isin(phase2city)].reset_index(drop=True)
# bubble_2.drop(bubble_2[(bubble_2.pc11_district_name == 'Aurangabad') & (bubble1.pc11_state_name == 'Bihar')].index, inplace=True)

In [125]:
bubble_2.loc[bubble_2[bubble_2['pc11_district_name'] == 'Thiruvananthapuram'].index, 'pc11_pca_tot_t'] = 1679754.0

### Phase 3

##### Phase 3 - Active cases (Oct-Nov-Dec 2020)

In [126]:
phase3dates = ['10-01','10-11','10-21','11-01','11-10','11-20','12-01','12-10','12-20','12-30']

In [127]:
phase3 = metrics_active_cases[metrics_active_cases['District'].isin(metrics_active_cases[(metrics_active_cases.State != metrics_active_cases.District) & (metrics_active_cases.District != 'Unknown') & (metrics_active_cases.District != 'Unknown (migrants and foreign returnees)')].loc[:,['State','District','10-01','10-11','10-21','11-01','11-10','11-20','12-01','12-10','12-20','12-30']].sort_values(by=['12-30'],ascending=False).head(10)['District'].tolist())].loc[:,['State','District','10-01','10-11','10-21','11-01','11-10','11-20','12-01','12-10','12-20','12-30']]

In [128]:
phase3 = phase3[['District','10-01','10-11','10-21','11-01','11-10'
                 ,'11-20','12-01','12-10','12-20','12-30']].melt(id_vars =['District'],
                                                        var_name = 'Phase3_Date',
                                                        value_name = 'Active_Case'
                                                       )

In [129]:
phase3['Phase3_Date'] =  '2020-'+ phase3.Phase3_Date 
phase3['Phase3_Date'] =  pd.to_datetime(phase3.Phase3_Date,format='%Y-%m-%d')

In [130]:
# display(HTML(phase2.to_html()))

In [131]:
phase3city = phase3.District.unique().tolist()
# merged_df[merged_df.pc11_district_name.isin(phase3city)]['pc11_state_name'].unique()
phase3states = ['Karnataka', 'Maharashtra','Kerala']

In [132]:
phase3copy = phase3.copy(deep=True)
phase3copy = phase3copy.set_index('Phase3_Date')
mobilitycopy3 = mobilitycopy.copy(deep=True)

In [133]:
# mobilitycopy3.date.unique().tolist()

In [134]:
mobilitycopy3['sub_region_2'] = mobilitycopy2['sub_region_2'].replace({'Bangalore Urban':'Bengaluru Urban','Mumbai':'Mumbai City'})
phase3mobility = mobilitycopy3[mobilitycopy3.sub_region_2.isin(phase3city)][['sub_region_2','date',
                                                         'grocery_and_pharmacy_percent_change_from_baseline']].query('(date>="2020-10-01") & (date<="2020-12-27")').reset_index(drop=True)

phase3mobility['date'] =  pd.to_datetime(phase3mobility.date,format='%Y-%m-%d')
phase3mobility = phase3mobility.rename(columns={'sub_region_2':'District','date':'Phase3_Date'})
phase3mobility = phase3mobility[phase3mobility.Phase3_Date.isin(phase3.Phase3_Date.unique().tolist())].reset_index(drop=True)
phase3mobility = phase3mobility.merge(phase3,how='left',on=['District','Phase3_Date']).reset_index(drop=True)

merged_df_3 = merged_df.copy(deep=True)
merged_df_3['pc11_district_name'] = merged_df_3['pc11_district_name'].replace({'Bangalore':'Bengaluru Urban','Mumbai':'Mumbai City'})
df_3 = merged_df_3[merged_df_3.pc11_district_name.isin(phase3city)][['pc11_state_name','pc11_district_name','pc11_pca_tot_t']].reset_index(drop=True)
df_3 = df_3.rename(columns={'pc11_district_name':'District','pc11_pca_tot_t':'total_population'})
# df1['District'] = df1['District'].replace({'Bangalore':'Bengaluru Urban','Mumbai':'Mumbai City', 'Ahmadabad':'Ahmedabad'})
# df_3.loc[df_3[df_3['District'] == 'Thiruvananthapuram'].index, 'total_population'] = 1679754.0

df_3.loc[df_3[df_3['District'] == 'Kozhikode'].index, 'total_population'] = 2028399.0
df_3.loc[df_3[df_3['District'] == 'Malappuram'].index, 'total_population'] = 1729522.0
df_3.loc[df_3[df_3['District'] == 'Thrissur'].index, 'total_population'] = 3121200.0
df_3.loc[df_3[df_3['District'] == 'Ernakulam'].index, 'total_population'] = 3282388.0
df_3.loc[df_3[df_3['District'] == 'Kottayam'].index, 'total_population'] = 1974551.0
df_3.loc[df_3[df_3['District'] == 'Pathanamthitta'].index, 'total_population'] = 1197412.0


df_3['total_population'] = df_3['total_population'].astype(np.int64)

phase3mobility = phase3mobility.merge(df_3,how='left',on=['District']).reset_index(drop=True)
phase3mobility['Phase3_Date'] = phase3mobility.Phase3_Date.dt.strftime('%Y-%m-%d')
phase3mobility = phase3mobility.dropna(subset=['total_population'])

del df_3
del merged_df_3

##### Taking data for third phase for percent postivity

In [135]:
df_3alt = percpositiveT.loc[percpositiveT.loc[:,'10-01':'12-31']['12-31'].sort_values(ascending=False).head(5).to_frame().index.tolist(), :].loc[:,'10-01':'12-31']

In [136]:
df_3 = percpositiveT.loc[percpositiveT.loc[:,'10-01':'12-31']['12-31'].sort_values(ascending=False).head(5).to_frame().index.tolist(), :].loc[:,'10-01':'12-31']

In [137]:
# sns.heatmap(df_2, cmap="YlGnBu", linewidths=0.5)

In [138]:
all_India = percpositiveT.iloc[percpositiveT.loc[:,'10-01':'12-31'].index == 'All INDIA'].loc[:,'10-01':'12-31']
df_3 = all_India.append(df_3)

In [139]:
df_3 = df_3.reset_index()
df_3 = df_3.melt(id_vars=['State'],var_name='Date',value_name='Test_Positivity')
# df1['Test_Positivity'] = df1.Test_Positivity.astype(int)
df_3['Date'] =  '2020-'+ df_3.Date
df_3['Date'] =  pd.to_datetime(df_3.Date,format='%Y-%m-%d')

##### Phase 3 - testing labs 

In [140]:
phase3states

['Karnataka', 'Maharashtra', 'Kerala']

In [141]:
totaldf_3= totaldf[phase3states].loc['2020-07-06':'2020-09-28',:].xs('All3',level=1).reset_index(level=0)

In [142]:
melt_3 = totaldf_3.melt(id_vars=['Date'],var_name='State',value_name='Testing_Labs')
melt_3['Date'] = melt_3['Date'].dt.strftime('%Y/%m/%d')
melt_3['Testing_Labs'] = melt_3.Testing_Labs.astype(int)

In [143]:
# # Static relploat
# sns.relplot(x="Date", y="Testing_Labs",
# data=melt1,
# kind="line",
# #style="sub_region_2",
# hue="State")

In [144]:
melt_3 = totaldf_3.melt(id_vars=['Date'],var_name='State',value_name='Testing_Labs')
melt_3['Testing_Labs'] = melt_3.Testing_Labs.astype(int)

##### Phase 3 state infrastructure

In [145]:
select_df_3 = select_df.copy(deep=True)
select_df_3['pc11_district_name'] = select_df_3['pc11_district_name'].replace({'Bangalore':'Bengaluru Urban','Mumbai':'Mumbai City'})

In [146]:
bubble_3 = select_df_3[select_df_3.pc11_district_name.isin(phase3city)].reset_index(drop=True)
# bubble_2.drop(bubble_2[(bubble_2.pc11_district_name == 'Aurangabad') & (bubble1.pc11_state_name == 'Bihar')].index, inplace=True)

In [147]:
# bubble_3.loc[bubble_3[bubble_3['pc11_district_name'] == 'Thiruvananthapuram'].index, 'pc11_pca_tot_t'] = 1679754.0

bubble_3.loc[bubble_3[bubble_3['pc11_district_name'] == 'Kozhikode'].index, 'pc11_pca_tot_t'] = 2028399.0
bubble_3.loc[bubble_3[bubble_3['pc11_district_name'] == 'Malappuram'].index, 'pc11_pca_tot_t'] = 1729522.0
bubble_3.loc[bubble_3[bubble_3['pc11_district_name'] == 'Thrissur'].index, 'pc11_pca_tot_t'] = 3121200.0
bubble_3.loc[bubble_3[bubble_3['pc11_district_name'] == 'Ernakulam'].index, 'pc11_pca_tot_t'] = 3282388.0
bubble_3.loc[bubble_3[bubble_3['pc11_district_name'] == 'Kottayam'].index, 'pc11_pca_tot_t'] = 1974551.0
bubble_3.loc[bubble_3[bubble_3['pc11_district_name'] == 'Pathanamthitta'].index, 'pc11_pca_tot_t'] = 1197412.0

### Phase 1 charts (Apr-May-Jun 2020)

**Phase 1 Analysis**

1. Top 10 districts were identified based on their active COVID-19 cases.
2. #### Active cases -
    - We can see a significant surge in the number of active cases in various districts of Maharashtra like
        Mumbai City, Thane, Pune.
    - By the end of phase 1, Chennai has the second highest number of active cases.
    - Most districts display a consistent rise in active cases, whereas two districts namely Hyderabad and Bengaluru Urban 
        show a steep rise around June 8th -9th. This trend may be due to Unlock 1 that started from June 8th.


In [148]:
fig = px.line(phase1, x="Phase1_Date", y="Active_Case", color="District",
               hover_name="District",
             labels = {
                     "Phase1_Date": "Date",
                     "Active_Case": "Active Cases"})
fig.update_layout({'title':{'text':
                           'Active Cases of Top Districts',
                           'x':0.45, 'y':0.95}
                     
                 })
fig.show()

In [149]:
for district in phase1copy.District.unique().tolist():
    layout_chart = cf.Layout(
                            height=800,
                            width=800
                    )
    
    phase1copy[phase1copy.District == district][['Active_Case']].iplot(
    mode='lines+markers',
    opacity=0.8,
    size=8,
    symbol=1,
    xTitle='Date',
    yTitle='Active Cases',
    title=district + ' Active Cases'
    #,layout=layout_chart.to_plotly_json()
    )
    

**Testing dynamics** 

Infrastructure and positivity rate moves in tandem with intensity of COVID-19 across Indian states/districts.

- Heat map analysis of test positivity rate of top states indicates that Maharashtra, Delhi had higher test positivity rate than their peer states. 
- Test positivity rate in Telangana picked up towards the end of phase-1 which coincides with the steep rise in active cases in one of its largest districts, Hyderabad. 


In [150]:
fig = go.Figure(data=go.Heatmap(
        z=df1.Test_Positivity,
        x=df1.Date,
        y=df1.State,
        colorscale='burgyl'))

fig.update_layout(
    title='Test Positivity',
    xaxis_nticks=30)

fig.show()

**State Infrastructure**


1. The scatter charts are depicted for top 10 districts (as per active case count). The size of the bubble indicates 
district population (2011 census). 
2. #### Scatter Chart 1 : District hospital penetration & age 60+ population share
      - The first scatter chart indicates that districts like Pune and Chennai that have high active cases and a relatively
        higher proportion of population in age 60+ years (a cohort that is potentially at a greater risk from COVID-19), do not have a high prevalence of district hospitals.
      - Cities like Mumbai which have a similar share of 60+ cohort but have a very high prevalence of district hospitals.
      - Low penetration of district hospitals, PHCs, CHCs and a high share of older population put certain districts at a 
        greater risk of the pandemic.


In [151]:
fig = px.scatter(bubble1,
                 x="60plus_share", y="pr_2011_dish",
	         size="pc11_pca_tot_t", color="pc11_district_name",
                 hover_name="pc11_district_name", size_max=60)
fig.update_layout({'title':{'text':
                           '60+ Age Share and District Hospitals Per 10,000 People',
                           'x':0.45, 'y':0.95}})
fig.update_xaxes(title_text='% 60+ Age Share of Population')
fig.update_yaxes(title_text='District Hospitals Per 10,000 People')
fig.show()

#### Scatter Chart 2: District hospital penetration & share of handwashing with soap

1. Most of the districts depict a higher proportion of households following handwashing with soap which is line 
with characteristics displayed by metropolitan cities.
2. Individual behaviour of handwashing, social distancing and wearing masks are equally important in combating the pandemic.


In [152]:
fig = px.scatter(bubble1,
                 x="hand_wash_soap", y="pr_2011_dish",
	         size="pc11_pca_tot_t", color="pc11_district_name",
                 hover_name="pc11_district_name", size_max=60)
fig.update_layout({'title':{'text':
                           '% Prevalence of Soap Hand Washing In Population and District Hospitals Per 10,000 Population',
                           'x':0.45, 'y':0.95}})
fig.update_xaxes(title_text='% Prevalence of Soap Hand Washing In Population')
fig.update_yaxes(title_text='District Hospitals Per 10,000 People')
fig.show()

#### Scatter Chart 3: District hospital penetration & percentage share of females with hypertension

1. Chennai and Pune show a high incidence of comorbidities like hypertension in their female population and 
lack of penetration of district hospitals (given their population).
2. Presence of comorbidities puts districts at a relatively greater risk.


In [153]:
fig = px.scatter(bubble1,
                 x="ex_htn_broad_ind_fem", y="pr_2011_dish",
	         size="pc11_pca_tot_t", color="pc11_district_name",
                 hover_name="pc11_district_name", size_max=60)
fig.update_layout({'title':{'text':
                           '% Prevalence of Hypertension in Female Population and District Hospitals Per 10,000 Population',
                           'x':0.45, 'y':0.95}})
fig.update_xaxes(title_text='Hypertension Instance Female (% of total population)')
fig.update_yaxes(title_text='District Hospitals Per 10,000 People')
fig.show()

##### Phase 1 mobility pattern across top cases districts

**Mobility pattern** seen in conjunction with active cases for top districts by active cases indicates two trends:

1. High active cases are followed by period of low mobility – either guided by state-specific restrictions or restraint 
observed by individuals. The reverse is also seen to hold true, wherein, low active cases may result in heightened mobility
on account of ease in restrictions.
2. In few cases, high mobility precedes high active cases and low mobility precedes low active cases
– supporting the conjecture that mobility restrictions help in containing active case count.


In [156]:
fig = px.scatter(phase1mobility, x="Active_Case",y="grocery_and_pharmacy_percent_change_from_baseline",
          animation_frame="Phase1_Date",animation_group="District",
          size="total_population", color="District", hover_name="District",size_max=50
            ,range_x=[0,30000], range_y=[-90,0])
# fig.update_layout(transition = {'duration': 50000})
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 1000
fig.update_xaxes(title_text='Active Cases')
fig.update_yaxes(title_text='% Change Mobility - Grocery')
fig.update_layout({'title':{'text':
                           'Mobility Change From Baseline',
                           'x':0.45, 'y':0.95}
                     
                 })
fig.show()

In [157]:
fig = px.line(phase1mobility, x="Phase1_Date", y="grocery_and_pharmacy_percent_change_from_baseline", color="District",
               hover_name="District",
             )
fig.update_layout({'title':{'text':
                           'Mobility Change From Baseline',
                           'x':0.45, 'y':0.95}
                     
                 })
fig.update_xaxes(title_text='Phase 1 Dates')
fig.update_yaxes(title_text='% Change Mobility - Grocery')
fig.show()

### Phase 2 charts (Jul-Aug-Sep 2020)

1. Active case trend is dominated by districts of `Maharashtra` even in phase 2. 
2. `Mumbai` and `Thane` show a V-shaped pattern in its active cases, reported on account of migration and ease in 
mobility restrictions.
3. `Chennai` shows a consistent decline in active cases in Phase 3.
d.	Many new states emerge as leading states with respect to test positivity, whereas `Maharashtra` remains in Phase 2 
with highest test positivity rate. Reasons for few states like `Tamil Nadu`, `Telangana` not featuring in 
top test positivity states could be:
    - Stabilisation in positive cases.
    - Higher testing relative to positive cases.
5. Test positivity rate is also seen to intensify as one moves through Phase 2.
6. Many cities that feature as the top districts by active cases are characterized by a high-risk combination of 
low district hospital penetration, high female hypertension share and high share of 60+ age in population.
7. Mobility pattern seen in conjunction with active cases for top districts by active cases indicates two trends:
    - High active cases are followed by period of low mobility – either guided by state-specific restrictions 
    or restraint observed by individuals.
    - In few cases, high mobility precedes high active cases and low mobility precedes low active cases
    – supporting the conjecture that mobility restrictions help in containing active case count.


In [158]:
fig = px.line(phase2, x="Phase2_Date", y="Active_Case", color="District",
               hover_name="District",
             labels = {
                     "Phase2_Date": "Date",
                     "Active_Case": "Active Cases"})
fig.update_layout({'title':{'text':
                           'Active Cases of Top Districts',
                           'x':0.45, 'y':0.95}
                     
                 })
fig.show()

In [159]:
for district in phase2copy.District.unique().tolist():
    layout_chart = cf.Layout(
                            height=800,
                            width=800
                    )
    
    phase2copy[phase2copy.District == district][['Active_Case']].iplot(
    mode='lines+markers',
    opacity=0.8,
    size=8,
    symbol=1,
    xTitle='Date',
    yTitle='Active Cases',
    title=district + ' Active Cases'
    #,layout=layout_chart.to_plotly_json()
    )
    

#### Phase 2 testing heatmap

In [160]:
fig = go.Figure(data=go.Heatmap(
        z=df_2.Test_Positivity,
        x=df_2.Date,
        y=df_2.State,
        colorscale='burgyl'))

fig.update_layout(
    title='Test Positivity',
    xaxis_nticks=30)

fig.show()

##### Phase 2 risk factors/state infrastructure

In [161]:
fig = px.scatter(bubble_2,
                 x="60plus_share", y="pr_2011_dish",
        size="pc11_pca_tot_t", color="pc11_district_name",
                 hover_name="pc11_district_name", size_max=60)
fig.update_layout({'title':{'text':
                           '60+ Age Share and District Hospitals Per 10,000 People',
                           'x':0.45, 'y':0.95}})
fig.update_xaxes(title_text='% 60+ Age Share of Population')
fig.update_yaxes(title_text='District Hospitals Per 10,000 People')
fig.show()

In [163]:
fig = px.scatter(bubble_2,
                 x="hand_wash_soap", y="pr_2011_dish",
         size="pc11_pca_tot_t", color="pc11_district_name",
                 hover_name="pc11_district_name", size_max=60)
fig.update_layout({'title':{'text':
                           '% Prevalence of Soap Hand Washing In Population and District Hospitals Per 10,000 Population',
                           'x':0.45, 'y':0.95}})
fig.update_xaxes(title_text='% Prevalence of Soap Hand Washing In Population')
fig.update_yaxes(title_text='District Hospitals Per 10,000 People')
fig.show()

In [164]:
fig = px.scatter(bubble_2,
                 x="ex_htn_broad_ind_fem", y="pr_2011_dish",
         size="pc11_pca_tot_t", color="pc11_district_name",
                 hover_name="pc11_district_name", size_max=60)
fig.update_layout({'title':{'text':
                           '% Prevalence of Hypertension in Female Population and District Hospitals Per 10,000 Population',
                           'x':0.45, 'y':0.95}})
fig.update_xaxes(title_text='Hypertension Instance Female (% of total population)')
fig.update_yaxes(title_text='District Hospitals Per 10,000 People')
fig.show()

##### Phase 2 mobility pattern across top cases districts

In [165]:
fig = px.scatter(phase2mobility, x="Active_Case",y="grocery_and_pharmacy_percent_change_from_baseline",
          animation_frame="Phase2_Date",animation_group="District",
          size="total_population", color="District", hover_name="District",size_max=50
            ,range_x=[0,75000], range_y=[-80,40])
# fig.update_layout(transition = {'duration': 50000})
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 1000
fig.update_xaxes(title_text='Active Cases')
fig.update_yaxes(title_text='% Change Mobility - Grocery')
fig.update_layout({'title':{'text':
                           'Mobility Change From Baseline',
                           'x':0.45, 'y':0.95}
                     
                 })
fig.show()

In [166]:
fig = px.line(phase2mobility, x="Phase2_Date", y="grocery_and_pharmacy_percent_change_from_baseline", color="District",
               hover_name="District"
             )
fig.update_layout({'title':{'text':
                           'Mobility Change From Baseline',
                           'x':0.45, 'y':0.95}
                     
                 })
fig.update_xaxes(title_text='Phase 2 Dates')
fig.update_yaxes(title_text='% Change Mobility - Grocery')
fig.show()

### Phase 3 charts (Oct-Nov-Dec 2020)

1. The top districts by active case count depict a declining trend consistent with the national averages.
2. Few districts of `Kerala` show a resurgence in active cases.
3. Heat map for test positivity rate shows decreasing intensity as we move towards the end of Phase 3.
4. Few districts of `Kerala` that have high active case count, show lower proportion of households following handwashing with soap and low penetration of district hospitals.
5. As active cases recede, mobility trends upward.


In [167]:
fig = px.line(phase3, x="Phase3_Date", y="Active_Case", color="District",
               hover_name="District",
             labels = {
                     "Phase3_Date": "Date",
                     "Active_Case": "Active Cases"})
fig.update_layout({'title':{'text':
                           'Active Cases of Top Districts',
                           'x':0.45, 'y':0.95}
                     
                 })
fig.show()

In [168]:
for district in phase3copy.District.unique().tolist():
    layout_chart = cf.Layout(
                            height=800,
                            width=800
                    )
    
    phase3copy[phase3copy.District == district][['Active_Case']].iplot(
    mode='lines+markers',
    opacity=0.8,
    size=8,
    symbol=1,
    xTitle='Date',
    yTitle='Active Cases',
    title=district + ' Active Cases'
    #,layout=layout_chart.to_plotly_json()
    )
    

#### Phase 3 testing heatmap

In [169]:
fig = go.Figure(data=go.Heatmap(
        z=df_3.Test_Positivity,
        x=df_3.Date,
        y=df_3.State,
        colorscale='burgyl'))

fig.update_layout(
    title='Test Positivity',
    xaxis_nticks=30)

fig.show()

##### Phase 3 risk factors/state infrastructure

In [170]:
fig = px.scatter(bubble_3,
                 x="60plus_share", y="pr_2011_dish",
	         size="pc11_pca_tot_t", color="pc11_district_name",
                 hover_name="pc11_district_name", size_max=40,
                 range_x=[0.015,0.11], range_y=[0.001,0.025])
fig.update_layout({'title':{'text':
                           '60+ Age Share and District Hospitals Per 10,000 People',
                           'x':0.45, 'y':0.95}})
fig.update_xaxes(title_text='% 60+ Age Share of Population')
fig.update_yaxes(title_text='District Hospitals Per 10,000 People')
fig.show()

In [171]:
fig = px.scatter(bubble_3,
                 x="hand_wash_soap", y="pr_2011_dish",
	         size="pc11_pca_tot_t", color="pc11_district_name",
                 hover_name="pc11_district_name", size_max=60)
fig.update_layout({'title':{'text':
                           '% Prevalence of Soap Hand Washing In Population and District Hospitals Per 10,000 Population',
                           'x':0.45, 'y':0.95}})
fig.update_xaxes(title_text='% Prevalence of Soap Hand Washing In Population')
fig.update_yaxes(title_text='District Hospitals Per 10,000 People')
fig.show()

In [172]:
fig = px.scatter(bubble_3,
                 x="ex_htn_broad_ind_fem", y="pr_2011_dish",
	         size="pc11_pca_tot_t", color="pc11_district_name",
                 hover_name="pc11_district_name", size_max=60)
fig.update_layout({'title':{'text':
                           '% Prevalence of Hypertension in Female Population and District Hospitals Per 10,000 Population',
                           'x':0.45, 'y':0.95}})
fig.update_xaxes(title_text='Hypertension Instance Female (% of total population)')
fig.update_yaxes(title_text='District Hospitals Per 10,000 People')
fig.show()

##### Phase 3 mobility pattern across top cases districts

In [174]:
fig = px.scatter(phase3mobility, x="Active_Case",y="grocery_and_pharmacy_percent_change_from_baseline",
          animation_frame="Phase3_Date",animation_group="District",
          size="total_population", color="District", hover_name="District",size_max=50
            ,range_x=[0,75000], range_y=[-40,60])
# fig.update_layout(transition = {'duration': 50000})
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 1000
fig.update_xaxes(title_text='Active Cases')
fig.update_yaxes(title_text='% Change Mobility - Grocery')
fig.update_layout({'title':{'text':
                           'Mobility Change From Baseline',
                           'x':0.45, 'y':0.95}
                     
                 })
fig.show()

In [175]:
fig = px.line(phase3mobility, x="Phase3_Date", y="grocery_and_pharmacy_percent_change_from_baseline", color="District",
               hover_name="District"
             )
fig.update_layout({'title':{'text':
                           'Mobility Change From Baseline',
                           'x':0.45, 'y':0.95}
                     
                 })
fig.update_xaxes(title_text='Phase 3 Dates')
fig.update_yaxes(title_text='% Change Mobility - Grocery')
fig.show()

### Effect of pollution on covid transmission

In [176]:
pollutiondf = merged_df[merged_df['mean_pollution'].notna()][['pc11_district_name','pc11_state_name','mean_pollution']]

In [177]:
district_poll = metrics_total_cases[~metrics_total_cases.District.isin(metrics_total_cases.State.unique().tolist())].loc[:,'District':]

In [178]:
district_poll1 = district_poll.query('District != "Unknown"').sort_values(by=['12-31'],ascending=False).head(40).reset_index(drop=True)
district_poll2 = district_poll.query('District != "Unknown"').sort_values(by=['12-31'],ascending=True).head(100).reset_index(drop=True)

In [179]:
district_pollk = pd.concat([district_poll1,district_poll2],ignore_index=True).reset_index(drop=True)

In [180]:
# Taking only top 100 districts by cumulative case counts as of 31st December 2020
district_poll = district_poll.query('District != "Unknown"').sort_values(by=['12-31'],ascending=False).head(100).reset_index(drop=True)

In [181]:
# Melt dataframe to get to required structure
district_pollk = district_pollk.melt(id_vars=['District'],var_name='Date',value_name='Cases')

In [182]:
# Melt dataframe to get to required structure
district_poll = district_poll.melt(id_vars=['District'],var_name='Date',value_name='Cases')

In [183]:
# del daily_cases
dflist = []
for district in district_poll.District.unique().tolist():
    district = district_poll[district_poll.District==district].reset_index(drop=True)
    daily_cases = [district.Cases[0]]
    for i in range(len(district.Date)):
        if i>=1:
            daily_cases.append(district.Cases[i]-district.Cases[i-1])
        district['Daily Cases'] = pd.Series(daily_cases)
    dflist.append(district)
            
    

In [184]:
polldata = pd.concat(dflist,ignore_index=True)

In [185]:
del daily_cases
dflistk = []
for district in district_pollk.District.unique().tolist():
    district = district_pollk[district_pollk.District==district].reset_index(drop=True)
    daily_cases = [district.Cases[0]]
    for i in range(len(district.Date)):
        if i>=1:
            daily_cases.append(district.Cases[i]-district.Cases[i-1])
        district['Daily Cases'] = pd.Series(daily_cases)
    dflistk.append(district)
            
    

In [186]:
polldatak = pd.concat(dflistk,ignore_index=True)

In [187]:
#  display(HTML(pune_c.to_html()))

In [188]:
pollutiondf['pc11_district_name'] = pollutiondf['pc11_district_name'].replace({'Allahabad':'(Allahabad) Pragyaraj',
                                                                               'Gautam Buddha Nagar':'Gautam Buddh Nagar',
                                                                               'Ahmadabad':'Ahmedabad',
                                                                               'Puducherry':'Pondicherry',
                                                                               'Kancheepuram':'Kanchipuram',
                                                                               'Belgaum':'Belagavi',
                                                                               'Dharwad':'Dharwada',
                                                                               'Bellary':'Ballari',
                                                                               'Shimoga':'Shivamogga',
                                                                               'Tumkur':'Tumakuru',
                                                                               'Bangalore':'Bengaluru Urban',
                                                                               'Hassan':'Hassana',
                                                                               'Mysore':'Mysuru',
                                                                                'Mumbai':'Mumbai City',
                                                                               'Ahmadnagar':'Ahmednagar',
                                                                                'Raigarh':'Raigad',
                                                                               'Gulbarga':'Kalaburagi',
                                                                               'North Twenty Four Parganas':'North 24 Parganas',
                                                                               'Haora':'Howrah',
                                                                               'Hugli':'Hooghly',
                                                                               'Gurgaon':'Gurugram'
                                                                              })

In [189]:
nfhs['pc11_district_name'] = nfhs['pc11_district_name'].replace({'Allahabad':'(Allahabad) Pragyaraj',
                                                                               'Gautam Buddha Nagar':'Gautam Buddh Nagar',
                                                                               'Ahmadabad':'Ahmedabad',
                                                                               'Puducherry':'Pondicherry',
                                                                               'Kancheepuram':'Kanchipuram',
                                                                               'Belgaum':'Belagavi',
                                                                               'Dharwad':'Dharwada',
                                                                               'Bellary':'Ballari',
                                                                               'Shimoga':'Shivamogga',
                                                                               'Tumkur':'Tumakuru',
                                                                               'Bangalore':'Bengaluru Urban',
                                                                               'Hassan':'Hassana',
                                                                               'Mysore':'Mysuru',
                                                                                'Mumbai':'Mumbai City',
                                                                               'Ahmadnagar':'Ahmednagar',
                                                                                'Raigarh':'Raigad',
                                                                               'Gulbarga':'Kalaburagi',
                                                                               'North Twenty Four Parganas':'North 24 Parganas',
                                                                               'Haora':'Howrah',
                                                                               'Hugli':'Hooghly',
                                                                               'Gurgaon':'Gurugram'
                                                                              })

In [190]:
pollutiondf = pollutiondf[['pc11_district_name','mean_pollution']].rename(columns={'pc11_district_name':'District'})
nfhs = nfhs[['pc11_district_name','ex_htn_broad_ind_fem']].rename(columns={'pc11_district_name':'District','ex_htn_broad_ind_fem':'female_hypertension'})

In [192]:
polldata = polldata.merge(pollutiondf,how='inner',on=['District']).reset_index(drop=True).merge(nfhs,how='inner',on=['District'])

In [194]:
polldata1 = polldata.copy(deep=True)
polldata1['Date'] =  '2020-'+ polldata1.Date
polldata1['Date'] =  pd.to_datetime(polldata1.Date,format='%Y-%m-%d')

In [195]:
polldata1 = polldata1.set_index("Date")

In [196]:
polldata['Date'] =  '2020-'+ polldata.Date
polldata['Date'] =  pd.to_datetime(polldata.Date,format='%Y-%m-%d')

In [197]:
polldata = polldata.dropna()
polldata1 = polldata1.dropna()

### Random Forest

In [198]:
polldata.sort_values(by="Date", ascending=True).head(10)

,District,Date,Cases,Daily Cases,mean_pollution,female_hypertension
0,Bengaluru Urban,2020-03-31,48.0,48.0,26.946268,9.222266
19044,Kanchipuram,2020-03-31,1.0,1.0,29.115577,20.411581
5244,Ahmednagar,2020-03-31,0.0,0.0,40.658252,13.353710
24564,Shivamogga,2020-03-31,0.0,0.0,20.955656,25.152576
5520,Anantapur,2020-03-31,2.0,2.0,26.855057,15.932012
5796,Raigad,2020-03-31,0.0,0.0,45.201096,17.825203
14628,Srikakulam,2020-03-31,0.0,0.0,31.644183,13.936093
5797,Raigad,2020-03-31,0.0,0.0,45.201096,13.217209
5798,Raigad,2020-03-31,0.0,0.0,39.573163,17.825203
5799,Raigad,2020-03-31,0.0,0.0,39.573163,13.217209


In [199]:
from sklearn.model_selection import train_test_split

from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

In [200]:
# Scale numeric values (standardScaler)
# One hot Encode categorical variables 
# Fill Empty values (SimpleImputer)

def preprocess_features(df_train, numeric_features, categorical_features, drop_features, target):

    all_features = set(numeric_features + categorical_features + drop_features + [target])
    print(all_features)
    if set(df_train.columns) != all_features:
        print("Missing columns", set(df_train.columns) - all_features)
        print("Extra columns", all_features - set(df_train.columns))
        raise Exception("Columns do not match")
    
    numeric_transformer = Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])  
    categorical_transformer = Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),#'constant', fill_value='?')),
        ('onehot', OneHotEncoder(sparse=False, drop='first'))#handle_unknown='ignore'))
    ])
    preprocessor = ColumnTransformer([
        ('numeric', numeric_transformer, numeric_features),
        ('categorical', categorical_transformer, categorical_features)
    ])
    preprocessor.fit(df_train);

    ohe = preprocessor.named_transformers_['categorical'].named_steps['onehot']
    ohe_feature_names = list(ohe.get_feature_names(categorical_features))
    new_columns = numeric_features + ohe_feature_names

    X_train_enc = pd.DataFrame(preprocessor.transform(df_train), index=df_train.index, columns=new_columns)
    y_train = df_train[target]
    
    return X_train_enc, y_train

In [201]:
numeric_features = ["mean_pollution", "female_hypertension"] 
categorical_features = ["District"]
drop_features = ["Date", "Daily Cases", "Cases"]
target = "Daily Cases"

df_train_enc, y_train = preprocess_features(polldata, 
        numeric_features,
        categorical_features, 
        drop_features,
        target)

{'Cases', 'mean_pollution', 'District', 'female_hypertension', 'Date', 'Daily Cases'}


In [202]:
# rf = RandomForestRegressor(n_estimators = 10)
# rf.fit(X_train_enc, y_train)

# rf_coefs = pd.DataFrame(data=rf.feature_importances_, index=X_train_enc.columns, columns=["Importance"])
# rf_coefs.abs().sort_values(by="Importance", ascending = False).head(5)



# Hyperparameter tuning using RandomizedSearchCV(Cross-Validation) for best results
from sklearn.model_selection import RandomizedSearchCV
import scipy.stats

rf_param_dist = {
              "n_estimators"     : scipy.stats.randint(low=5, high=15),
              "max_depth"        : scipy.stats.randint(low=5, high=10),
              "max_features"     : scipy.stats.randint(low=1, high=10)
             }

rf_search = RandomizedSearchCV(RandomForestRegressor(random_state=123), rf_param_dist, return_train_score=True, verbose=1, n_iter=20, cv=3, random_state=123)
rf_search.fit(df_train_enc, y_train);

# Display Random Forest's Feature Importances
rf_search.best_estimator_.feature_importances_
rf_coefs = pd.DataFrame(data=rf_search.best_estimator_.feature_importances_, index=df_train_enc.columns, columns=["Importance"])
rf_coefs.sort_values(by="Importance", ascending = False).head(5)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:    5.7s finished


,Importance
District_Bengaluru Urban,0.269724
District_Pune,0.248689
mean_pollution,0.123206
female_hypertension,0.117737
District_Thane,0.084601


In [203]:
rf = RandomForestRegressor(n_estimators = 10)
rf.fit(df_train_enc, y_train)
rf.score(df_train_enc, y_train)

0.1761166125320931

# K-Means Clustering

**Methodology**


- Unsupervised machine learning algorithm.
- Two types of machine learning algorithms – 
    * Supervised Learning 
    * Unsupervised Learning 
- Clustering - Each data observation may be characterized by several attributes called features. 
    Clustering algorithms aim at clubbing data points into clusters bases on similarity/patterns among these features
    or attributes.
- Feature selection 
- How does it work – Centroid based algorithm 
- Optimal number of cluster selection criterion:
`The Elbow Method`
- Performance evaluation of K-means clustering:
`Dunn Index`

$Dunn Index=⁡(Inter Cluster Distance)_{min}/(Intra Cluster Distance)_{max}⁡$

    * Higher the value of Dunn Index better is the clustering.
    * Lower the inertia better is the clustering.



***

**Clustering Framework** 


1.Features Selected
   - Total cases as of 30th September 2020
   - Number of district hospitals per ten thousand population in the district
   - Age distribution:
       - Percentage of district population with age 60+
       - Percentage of households with a 65+ member
   - Mean pollution
   - Average number of members per room – used as a proxy for population density
   - Percentage households using soap for handwashing

2.Features were normalized and missing values were removed.

3.Approximately 100 districts were chosen based on fill rate.

***

In [204]:
from scipy.cluster.vq import whiten
from scipy.cluster.vq import kmeans, vq
from numpy import random

In [205]:
random.seed(11)

In [206]:
polldata = pd.concat(dflist,ignore_index=True)

In [207]:
polldata = polldata.merge(pollutiondf,how='inner',on=['District']).reset_index(drop=True).merge(nfhs,how='inner',on=['District'])

In [208]:
df = polldata[polldata.Date == '09-30'].reset_index(drop=True)

In [209]:
merged_df['pc11_district_name'] = merged_df['pc11_district_name'].replace({'Allahabad':'(Allahabad) Pragyaraj',
                                               'Gautam Buddha Nagar':'Gautam Buddh Nagar','Puducherry':'Pondicherry',
                                              'Kancheepuram':'Kanchipuram','Belgaum':'Belagavi','Dharwad':'Dharwada','Bellary':'Ballari',
                            'Shimoga':'Shivamogga','Tumkur':'Tumakuru','Bangalore':'Bengaluru Urban','Hassan':'Hassana',
                               'Mysore':'Mysuru','Mumbai':'Mumbai City','Ahmadnagar':'Ahmednagar','Raigarh':'Raigad','Gulbarga':'Kalaburagi',
                    'North Twenty Four Parganas':'North 24 Parganas','Haora':'Howrah','Hugli':'Hooghly','Gurgaon':'Gurugram'
                                                                              })
merged_df = merged_df[['pc11_district_name',
       'pc11_state_name','hand_wash_water', 'hand_wash_soap', 'mem_per_room',
       'dum_65p','ex_diab_broad_ind_fem','bmigrt27_5_fem', 'csmoke_fem',
                       'instmigrationrate','outstmigrationrate','pr_2011_dish',
                       'pc11_pca_tot_t', 'pc11_pdensity','pc11_pca_tot_p','60plus_share'
                      ]].rename(columns={'pc11_district_name':'District','pc11_state_name':'State',
                                        'ex_diab_broad_ind_fem':'female_diabetic','bmigrt27_5_fem':'female_migraine',
                                         'pc11_pca_tot_t':'total_population','pc11_pdensity':'population_density'
                                        })

In [210]:
df = df.merge(merged_df,how='inner',on=['District']).reset_index(drop=True)

In [211]:
# def Repeat(x): 
#     _size = len(x) 
#     repeated = [] 
#     for i in range(_size): 
#         k = i + 1
#         for j in range(k, _size): 
#             if x[i] == x[j] and x[i] not in repeated: 
#                 repeated.append(x[i]) 
#     return repeated 

In [212]:
# print(Repeat(df[['District','State']].drop_duplicates().District.tolist()))

In [213]:
df.drop(df[((df.District == 'Aurangabad') & (df.State == 'Bihar')) | (df.District == 'Raigad')].index, inplace=True)

In [214]:
# df.drop(df[((df.District == 'Aurangabad') & (df.State == 'Bihar')) | ((df.District == 'Raigad') & (df.State == 'Chhattisgarh'))].index, inplace=True)

In [215]:
polldatak = polldatak.merge(pollutiondf,how='inner',on=['District']).reset_index(drop=True).merge(nfhs,how='inner',on=['District'])

In [216]:
dfk = polldatak[polldatak.Date == '09-30'].reset_index(drop=True)

In [217]:
dfk = dfk.merge(merged_df,how='inner',on=['District']).reset_index(drop=True)

In [218]:
dfk.drop(dfk[((dfk.District == 'Aurangabad') & (dfk.State == 'Bihar')) | (dfk.District == 'Raigad') ].index, inplace=True)

In [219]:
dfk1= dfk.copy(deep=True)
dfk1 = dfk1[dfk1[['District','pr_2011_dish','total_population']]['total_population'].notna()]
colsk = ['Cases', 'Daily Cases', 'mean_pollution',
       'female_hypertension','hand_wash_water', 'hand_wash_soap',
       'mem_per_room', 'dum_65p', 'female_diabetic', 'female_migraine',
       'csmoke_fem','pr_2011_dish','total_population','60plus_share']
dfk1_new = dfk1[['District']]

In [220]:
for col in colsk:
    result = []
    for i in range(len(dfk1[col])):
        result.append(whiten(dfk1[col]).tolist()[i])
    dfk1_new[col] = result

In [221]:
df = dfk1_new

In [222]:
# distortions = []
# num_clusters = range(2,7)
# # Getting distortios for various clusters
# for i in num_clusters:
#     centroids, distortion = kmeans(df[['Cases','total_population','pr_2011_dish',
#                                        'hand_wash_soap','60plus_share',
#                                        'mem_per_room','dum_65p','mean_pollution']],i)
#     distortions.append(distortion)

In [223]:
# # Plotting elbow plot data
# elbow_plot_data = pd.DataFrame({'num_clusters': num_clusters,
# 'distortions': distortions})
# sns.lineplot(x='num_clusters', y='distortions',
# data = elbow_plot_data)
# plt.show()

#### Fitting model and generating clusters

In [224]:
# Generate cluster centers and labels
cluster_centers, _ = kmeans(df[['Cases','pr_2011_dish',
                                       '60plus_share',
                                       'dum_65p','mean_pollution','mem_per_room', 'hand_wash_water']], 2)
df['cluster_labels'], _ = vq(df[['Cases','pr_2011_dish',
                                       '60plus_share',
                                       'dum_65p','mean_pollution','mem_per_room', 'hand_wash_water']], cluster_centers)

In [225]:
# # Plot clusters
# sns.scatterplot(x='total_population', y='Cases', hue='cluster_labels', data=df)
# plt.show()

In [226]:
dfbar = df.groupby('cluster_labels')['Cases','pr_2011_dish',
                                       '60plus_share',
                                       'dum_65p','mean_pollution','mem_per_room', 'hand_wash_water'].mean()

***

**Results** 


1. Two district clusters were created:
   - Cluster 0 – Low risk cluster
       - Low prevalence of cases
       - Higher penetration of district hospitals
       - Lower proportion of age 60+ population
       - Lower mean pollution 
       - Lower number of members per room
            
  - Cluster 1 – High risk cluster
       - High prevalence of cases
       - Lower penetration of district hospitals
       - Higher proportion of age 60+ population
       - Higher mean pollution 
       - Higher number of members per room


2. Handwashing habits appear similar in both the clusters.
3. Population density, health infrastructure, age profile are differentiating features between the two clusters.


***

In [227]:
df.groupby('cluster_labels')['Cases','pr_2011_dish',
                                       '60plus_share',
                                       'dum_65p','mean_pollution',
                             'mem_per_room', 'hand_wash_water'].mean().iplot(kind='bar',
                                                                             xTitle='Cluster Labels',yTitle='Variable Mean',
    title='Comparison Chart For Cluster Labels')

In [228]:
df[df.cluster_labels == 0]['District'].unique()

array(['Gurugram', 'Karbi Anglong', 'Bongaigaon', 'Baksa', 'Longleng',
       'Nalbari', 'Sivasagar', 'Jorhat', 'Saiha', 'New Delhi', 'Chirang',
       'Sonitpur', 'Wokha', 'Kiphire', 'Chandel', 'Phek', 'Goalpara',
       'Hailakandi', 'Morigaon', 'Dibrugarh', 'Dhemaji', 'Udalguri',
       'Lakhimpur', 'Kokrajhar', 'Karimganj', 'Tinsukia', 'Mamit',
       'Darrang', 'Dima Hasao', 'Barpeta', 'Cachar', 'Bishnupur',
       'Kurung Kumey'], dtype=object)

In [229]:
df[df.cluster_labels == 1]['District'].unique()

array(['Bengaluru Urban', 'Pune', 'Mumbai City', 'Thane', 'Chennai',
       'Nagpur', 'East Godavari', 'Kolkata', 'North 24 Parganas',
       'Nashik', 'West Godavari', 'Chittoor', 'Lucknow', 'Guntur',
       'Ahmednagar', 'Anantapur', 'Sri Potti Sriramulu Nellore',
       'Prakasam', 'Kurnool', 'Visakhapatnam', 'Jaipur', 'Jalgaon',
       'Indore', 'Satara', 'Solapur', 'Khordha', 'Raipur', 'Mysuru',
       'Coimbatore', 'North Goa', 'Medak', 'Khammam', 'Nalgonda',
       'Adilabad', 'Karimnagar', 'Nizamabad', 'Champhai'], dtype=object)

In [230]:
fig = px.scatter(df,
                 x="pr_2011_dish", y="hand_wash_soap",
	         size="total_population", color="cluster_labels",
                 hover_name="District", size_max=40
                ,range_x=[0,0.4], range_y=[0,5.5])
fig.update_layout({'title':{'text':
                           'Clusters',
                           'x':0.45, 'y':0.95}})
fig.update_xaxes(title_text='District Hospitals Prevalence Per 10,000 People')
fig.update_yaxes(title_text='% Prevalence of Hand Washing Using Soap')
fig.show()

In [231]:
dfc = df.copy()
dfc['cluster_labels'] = dfc['cluster_labels'].astype(str)

In [232]:
dfc.iplot(x='pr_2011_dish', y='hand_wash_soap', z='60plus_share', kind='bubble3d', xTitle='% District Hospitals', yTitle='Soap Hand Wash',
         zTitle='60+ Population', theme='pearl', size='total_population',
         categories='cluster_labels', title='3D Bubble Plot Sized by Population and Colored by Cluster Labels')

# Kriging Model

**Methodology**

     1. Geospatial interpolation model.
     2. Model understanding:
        - Consider a certain geography with each location characterized by its geo-coordinates 
            and a particular attribute (which we may want to predict, say elevation). 
            To articulate the above thought with the aid of a model, we may characterize xi 
            as the geographic location of a point ‘i’ (say latitude/longitude) and yi as 
            its specific attribute, say elevation.
        - The Kriging model states that attribute (in our case, elevation) for a location 
            say ‘new’ can be expressed as a linear combination of attributes of other locations 
            that are geographically proximate to location ‘new’.
        - Therefore, we can express ynew as a linear combinate of attributes of 5 of its proximate 
            geographic locations (for simplicity taking into account 5 points in close vicinity):


`ynew = wT y + Error 
ynew = w1y1 + w2y2 + w3y3 + w4y4 + w5y5 + Error
`

Where the wi’s indicate the weights assigned to each location.

***
The Kriging model thus predicts ynew as

`ynew pred = w1y1 + w2y2 + w3y3 + w4y4 + w5y5` ; where the respective weights are derived from the variogram (explained ahead)


***

#### Variogram:
    
Variograms indicate the relation between the distance between two geographic locations and the corresponding
deviation between their respective attributes.

Mathematically, we can write it as a function $γ(h):
γ(h) =  (1/2) [y_{i} - y_{j}]^{2}$ ; where ‘h’ is the distance between location $_{i}$ and $_{j}$.   $ h = x_{i}-x_{j} $


Solving for weights wi’s, is dependent on solving the following matrix equation: 

$A_{w} = B$
In view of the kriging model structure highlighted above, ‘A’ is the matrix of variogram functions
between $x_{i}$ and $x_{j}$ (all 5 known locations used to model ‘new’ location). 
‘B’ is the matrix of variogram functions between 
each of the 5 $x_{i}$’s and $x_{new}$. ‘w’ is matrix of weights. Therefore, the weights can be solved through the following matrix inversion:

$w= A^{-1}B$


These estimated weights can then be plugged in the kriging model to obtain:
`ynew pred = w1y1 + w2y2 + w3y3 + w4y4 + w5y5`


***
Assumptions for the application of the Kriging Model:
1. Stationarity
2. Constant Variogram
***

**Modeling Framework**

- Model trained on districts of India other than Delhi.
- The centroid latitudes and longitudes of the respective districts were fed as the location attribute. The COVID-19 cases were considered for the spatial characteristic to be predicted.
- The model was tested on Delhi’s districts centroid latitudes and longitudes.


In [233]:
import pykrige.kriging_tools as kt
from pykrige.ok import OrdinaryKriging

In [234]:
d_centroids = pd.read_csv('./Data/district_wise_centroids.csv')
s_centroids = pd.read_csv('./Data/state_wise_centroids_2001.csv') 
s_centroids = s_centroids.rename(columns={'Longitude':'S_Longitude','Latitude':'S_Longitude'})
centroids = d_centroids.merge(s_centroids, how='inner',on=['State']).reset_index(drop=True)

In [235]:
orig = metrics_active_cases[(metrics_active_cases.State != metrics_active_cases.District) & (metrics_active_cases.District != 'Unknown') & (metrics_active_cases.District != 'Unknown (migrants and foreign returnees)')]

In [236]:
delhi_c = centroids[centroids.State == 'Delhi'].reset_index(drop=True)

In [237]:
df0 = pd.DataFrame(columns=['State','District','Latitude','Longitude','S_Longitude','S_Longitude'])
df0.loc[0] = ['Delhi','Central Delhi','28.6643','77.2167','77.128045','28.645944']
df0.loc[1] = ['Delhi','East Delhi','28.6280','77.2956','77.128045','28.645944']
df0.loc[2] = ['Delhi','New Delhi','28.6139','77.2090','77.128045','28.645944']
df0.loc[3] = ['Delhi','North Delhi','28.7886','77.1412','77.128045','28.645944']
df0.loc[4] = ['Delhi','North East Delhi','28.7184','77.2580','77.128045','28.645944']
df0.loc[5] = ['Delhi','North West Delhi','28.7186','77.0685','77.128045','28.645944']
df0.loc[6] = ['Delhi','Shahdara','28.6894','77.2919','77.128045','28.645944']
df0.loc[7] = ['Delhi','South Delhi','28.4817','77.1873','77.128045','28.645944']
df0.loc[8] = ['Delhi','South East Delhi','28.5630','77.2611','77.128045','28.645944']
df0.loc[9] = ['Delhi','South West Delhi','28.5929','77.0346','77.128045','28.645944']
df0.loc[10] = ['Delhi','West Delhi','28.6663','77.0680','77.128045','28.645944']

In [238]:
delhi_c = pd.concat([delhi_c,df0],ignore_index=True).reset_index(drop=True)
delhi_c = delhi_c.query('State != District')
delhi_c = delhi_c.merge(orig,how='inner',on=['District']).reset_index(drop=True)
delhi_c = delhi_c.drop(['State_y'],axis=1).rename(columns={'State_x':'State'})
delhi_c1 = delhi_c[['State', 'District', 'Latitude','Longitude','S_Longitude','S_Longitude','06-30']]
# delhi_c1c['06-30'] = delhi_c1c['06-30'].astype(float)
delhi_c1['Latitude'] = delhi_c1['Latitude'].astype(float)
delhi_c1['Longitude'] = delhi_c1['Longitude'].astype(float)

In [239]:
centroids = centroids.merge(orig,how='inner',on=['District','State']).reset_index(drop=True)
centroids1 = centroids[['State', 'District', 'Latitude','Longitude','06-30']]

#### Ordinary Kriginig model using linear variogram

In [240]:
OK = OrdinaryKriging(
    x=np.array(centroids1.iloc[:, 2]),
    y=np.array(centroids1.iloc[:, 3]),
    z=np.array(centroids1.iloc[:, 4]),
    variogram_model="linear",
    verbose=False,
    enable_plotting=False,
)

In [241]:
gridx = np.array(delhi_c1.iloc[:, 2])
gridy = np.array(delhi_c1.iloc[:, 3])

In [242]:
z, ss = OK.execute("points", gridx, gridy)

In [243]:
# # # kt.write_asc_grid(gridx, gridy, z, filename="output.asc")
# plt.imshow(z)
# plt.show()

In [244]:
x=np.array(centroids1.iloc[:, 2])
y=np.array(centroids1.iloc[:, 3])

In [245]:
z.compressed().tolist()

[158.10306752530292,
 159.01684103706302,
 163.0695672037086,
 148.6296722711359,
 151.84154882006746,
 155.73235449256765,
 153.318259158222,
 176.3179697656845,
 166.6212899697644,
 166.66346852796843,
 160.25962382562545]

**Results**

- A pattern was observed in the case prediction for Delhi, wherein, the districts of other states that share a boundary
    with Delhi or are contiguous had a higher weight (than the districts that belong to non-contiguous states) 
    and the pattern in their Covid-19 case counts was reflected in the pattern for different districts of Delhi.
- These spatial interpolations become more accurate when a very proximate geography is considered, 
    which require rich latitude and longitude data for different points within a geography.
- Spatial interpolations can help policy makers predict different variables where the underlying geographies
    exhibit spatial correlation and thus make geography-specific interventions.


#### Ratio chart - actual vs predicted for Kringed predicted data for Delhi districts

In [246]:
pd.DataFrame({'Actual':delhi_c1['06-30'].tolist(),'Predicted':z.compressed().tolist()}).iplot(
    kind='spread', mode='markers', yTitle='Number', title='Spread between Actual and Predicted Covid19 cases for Delhi')